In [3]:
import os
import sys
sys.path.append('/home/ubuntu/')

import numpy as np
import pandas as pd
from litellm import completion
import os
from dotenv import load_dotenv
import nltk
# nltk.download('words')
from nltk.corpus import words
import enchant
import re
from tqdm import tqdm
from type import Trajectory, Word

d = enchant.Dict("en_US")

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

df = pd.read_csv('Wordle Words.csv')
filtered_df = df.dropna().sort_values(by='occurrence', ascending=False)

model_name = 'gpt-4o'
custom_llm_provider='openai'
api_base = "https://api.openai.com/v1"


# model_name = 'hosted_vllm/Qwen2.5-7B-Instruct'
# custom_llm_provider = 'vllm'
# api_base = "http://localhost:8000/v1"

In [2]:
def get_feedback(word, guess, d):
    """
    trajectory: list of tuples (word, result)
    word: string
    guess: string
    """
    if not d.check(guess) and not guess in words.words():
        return "Invalid word, not a valid English word"
    
    if len(guess) != 5:
        return "Invalid word, not a 5 letter word"
    
    response = ""
    response_text = ""
    for i, alphabet in enumerate(guess):
        if word[i] == alphabet:
            response += "G"
            response_text += f'{alphabet} is in the word and in the correct position.\n'
        elif alphabet in word:
            response += "Y"
            response_text += f'{alphabet} is in the word but in the wrong position.\n'
        else:
            response += "B"
            response_text += f'{alphabet} is not in the word.\n'
    return f"{guess} -> {response}"
    # return f"{guess} -> {response}\n{response_text}"

print(get_feedback("hello", "tells", d))

tells -> BGGGB


In [3]:
system_prompt = """
You are a Wordle solver assistant. Your goal is to correctly guess the hidden 5-letter word within six attempts. The game provides feedback after each guess, which you must use to refine your subsequent guesses. Follow these rules and strategies carefully:

== Wordle Rules ==
1. The target word is exactly 5 letters long and is a valid English word.
2. After each guess, you will receive feedback consisting of a string of 5 characters using the letters:
   - G (Green): Correct letter in the correct position.
   - Y (Yellow): Correct letter in the wrong position.
   - B (Black): Letter is not in the target word at all.
3. Use the feedback to eliminate invalid words and improve future guesses.

== Feedback Interpretation Examples ==
- Target word: "hello"
- Guess: "world"
- Feedback: "BYBGB"
  Explanation:
    - "w" → B: not in the target word
    - "o" → Y: in the word, but not in position 2
    - "r" → B: not in the word
    - "l" → G: correct letter and position (index 3)
    - "d" → B: not in the word

== Your Responsibilities ==
- Generate a single valid 5-letter English word for each turn.
- Use previous guesses and their feedback to eliminate impossible candidates.
- Prioritize guesses that test multiple hypotheses (e.g., new letters or positions) when unsure.
- Think step-by-step to explain your reasoning before producing each guess.

== Output Format ==
Your response must follow this structure:
<think>Explain your reasoning and deduction based on prior guesses and feedback.</think><answer>guess</answer>

- The text inside <think> and </think> should include your full reasoning step by step. Do not ignore this step.
- The word inside <answer> and </answer> must be your final guess.
- The guess must be a single valid 5-letter English word in lowercase only.
- Do not include any other text outside of the <think> and <answer> tags.

== Goal ==
Correctly identify the hidden word using logical deduction and strategic guessing, with clear and structured reasoning output.
"""

In [4]:
import re
def parser(text):
    reasoning = re.search(r"<think>(.*?)</think>", text, re.DOTALL)
    guess = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    
    if reasoning and guess:
        return reasoning.group(1).strip(), guess.group(1).strip()
    elif not reasoning and guess:
        return None, guess.group(1).strip()
    elif reasoning and not guess:
        return reasoning.group(1).strip(), None
    else:
        return None, None

In [5]:
import hashlib
def hash_word(s: str, bits=16):
    hash_obj = hashlib.sha256(s.encode('utf-8'))
    return int(hash_obj.hexdigest(), 16) % (2 ** bits)
print(hash_word("wordle"))
print(hash_word("Wordle"))

40325
43032


In [7]:
sft_df = filtered_df.sample(500)
words = []
for idx, row in sft_df.iterrows():
    df_index = idx
    word = row['word']
    occurrence = row['occurrence']
    hash = hash_word(word)
    W = Word(
        df_index=df_index,
        hash=hash,
        word=word,
        occurrence=occurrence,
    )
    words.append(W)

def word_to_str(word: Word, indent: int = 4) -> str:
    prefix = ' ' * indent
    # Convert each field into a formatted key=value string
    fields = [
        f"{k}={repr(v)}" 
        for k, v in word.model_dump().items() 
        if v is not None
    ]
    return prefix + "Word(" + ", ".join(fields) + ")"

# Build the content list with indentation
lines = [word_to_str(word, indent=4) for word in words]

# Join into a full Python source file
file_content = (
    "from type import Word\n\n"
    "TRAIN_WORDS = [\n" +
    ",\n".join(lines) +
    "\n]"
)

# Write the content to file
with open("/home/ubuntu/Wordle/Data/train.py", "w") as f:
    f.write(file_content)

In [ ]:

print('WORDS: ', chosen_words)
cost = 0.0

total_games = 0
games_won = 0


for i, word in tqdm(enumerate(chosen_words)):
    trajectory = [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': 'Your first guess is?'}
                ]
    print('New Word: ', word)
    W = Word(
        id=
        word=word,
    )
    total_games += 1
    
    for j in range(6):
        response = completion(model='o3', custom_llm_provider='openai', messages=trajectory)
        # response = completion(model=model_name, api_base=api_base, messages=trajectory)
        # Get the answer and reasoning, and add it to the trajectory
        answer = response.choices[0].message.content
        # Get the guess from the answer and get feedback
        reasoning, guess = parser(answer)

        if 'reasoning_content' in vars(response.choices[0].message):
            reasoning = response.choices[0].message.reasoning_content
            trajectory.append({'role': 'assistant', 'content': f'<think>{reasoning}</think><answer>{guess}</answer>'})
        else:
            trajectory.append({'role': 'assistant', 'content': answer})

        if response._hidden_params["response_cost"] is not None:
            cost += response._hidden_params["response_cost"]
        
        
        feedback = None
        if guess:
            feedback = get_feedback(word, guess, d)
            trajectory.append({'role': 'user', 'content': feedback})
        else:
            trajectory.append({'role': 'user', 'content': f'<think>Invalid format, stick to the <think>, </think> and <answer>, </answer> tags provided in the system prompt'})
        
        # If the feedback is 'GGGGG', break
        print(f'{reasoning} {guess}')
        if feedback.split('->')[-1].strip() == 'GGGGG':
            print(f'Word {word} found in {j+1} turns')
            games_won += 1
            trajectory[-1]['content'] = f'Success! You found the word {word} in {j+1} turns.'
            break

    # print(trajectory)
print('Total cost: ', cost)
print('Total games: ', total_games)
print('Games won: ', games_won)
print('Win rate: ', games_won/total_games)

In [23]:
import json
import os

# Write the word to words-used.json
# Read existing data from words-used.json
with open('Results/words-used.json', 'r') as f:
    existing_words = json.load(f)

# Append new word and write back
existing_words.append(word)

# Create the content with variable name
content = f"USED_WORDS = {json.dumps(existing_words, indent=2)}"

with open('Results/words-used.json', 'w') as f:
    f.write(content)

In [ ]:
# Read existing data from trajectories.json
try:
    with open('Results/trajectories.json', 'r') as f:
        existing_trajectories = json.load(f)
    if not isinstance(existing_trajectories, list):
        existing_trajectories = [existing_trajectories]
except (FileNotFoundError, json.JSONDecodeError):
    existing_trajectories = []

# Append new trajectory and write back
existing_trajectories.append(trajectory)
with open('Results/trajectories.json', 'w') as f:
    json.dump(existing_trajectories, f, indent=4)


In [ ]:
import requests

# Replace with your vLLM server URL
VLLM_SERVER_URL = "http://localhost:8000"

def get_model_info():
    try:
        response = requests.get(f"{VLLM_SERVER_URL}/v1/models")
        response.raise_for_status()
        models = response.json().get("data", [])
        if not models:
            print("No models are currently loaded.")
        else:
            print("Loaded models:")
            for model in models:
                print(f"- ID: {model.get('id')} | Owned by: {model.get('owned_by', 'unknown')}")
    except requests.RequestException as e:
        print(f"Error querying vLLM server: {e}")

if __name__ == "__main__":
    get_model_info()

Loaded models:
- ID: MERGED_WORDLE_UnslothFineTune_Qwen2.5-7B_Run3_16bit_r256_alpha256 | Owned by: vllm


In [4]:
from Wordle.Data.train import TRAIN_WORDS

In [5]:
from datasets import Dataset

In [9]:
word_dicts = [word.model_dump() for word in TRAIN_WORDS]
ds = Dataset.from_list(word_dicts)

In [17]:
type(ds[0])

dict

In [16]:
[Word(**item) for item in ds[0:1]]

TypeError: type.Word() argument after ** must be a mapping, not str

In [18]:
Word(**ds[0])

Word(df_index=5388, hash=20243, word='inlet', occurrence=6.439310127461795e-06, linked_trajectories=[], games_won=0, games_total=0)

In [20]:
ds = Dataset.from_list([Word(**item.model_dump()) for item in TRAIN_WORDS])

AttributeError: 'Word' object has no attribute 'get'

In [21]:
type(d)

enchant.Dict

In [23]:
ds[1]

{'df_index': 12889,
 'hash': 17684,
 'word': 'zebra',
 'occurrence': 6.835005632410685e-07,
 'linked_trajectories': [],
 'games_won': 0,
 'games_total': 0}

In [1]:
import json

content = json.load(open('/home/ubuntu/Wordle/Results/trajectories.json'))

In [25]:
content[0]

{'word': 'zebra',
 'word_hash': 17684,
 'messages': [{'role': 'system',
   'content': 'You are a Wordle solver assistant. Your goal is to correctly guess the hidden 5-letter word within six attempts. The game provides feedback after each guess, which you must use to refine your subsequent guesses. Follow these rules and strategies carefully:\n\n== Wordle Rules ==\n1. The target word is exactly 5 letters long and is a valid English word.\n2. After each guess, you will receive feedback consisting of a string of 5 characters using the letters:\n   - G (Green): Correct letter in the correct position.\n   - Y (Yellow): Correct letter in the wrong position.\n   - B (Black): Letter is not in the target word at all.\n3. Use the feedback to eliminate invalid words and improve future guesses.\n\n== Feedback Interpretation Examples ==\n- Target word: "hello"\n- Guess: "world"\n- Feedback: "BYBGB"\n  Explanation:\n    - "w" → B: not in the target word\n    - "o" → Y: in the word, but not in positi

In [14]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
qualified_trajectories = [T for T in content if T['solved'] == True]
train_trajectories, test_trajectories = train_test_split(qualified_trajectories, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_trajectories)
test_dataset = Dataset.from_list(test_trajectories)

In [15]:
content[0]

{'word': 'inlet',
 'word_hash': 20243,
 'messages': [{'role': 'system',
   'content': 'You are a Wordle solver assistant. Your goal is to correctly guess the hidden 5-letter word within six attempts. The game provides feedback after each guess, which you must use to refine your subsequent guesses. Follow these rules and strategies carefully:\n\n== Wordle Rules ==\n1. The target word is exactly 5 letters long and is a valid English word.\n2. After each guess, you will receive feedback consisting of a string of 5 characters using the letters:\n   - G (Green): Correct letter in the correct position.\n   - Y (Yellow): Correct letter in the wrong position.\n   - B (Black): Letter is not in the target word at all.\n3. Use the feedback to eliminate invalid words and improve future guesses.\n\n== Feedback Interpretation Examples ==\n- Target word: "hello"\n- Guess: "world"\n- Feedback: "BYBGB"\n  Explanation:\n    - "w" → B: not in the target word\n    - "o" → Y: in the word, but not in positi

In [17]:
from unsloth import FastLanguageModel, is_bfloat16_supported, apply_chat_template
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForSeq2Seq
from typing import Optional
import torch

def load_data(dataset_path = '/home/ubuntu/Wordle/Results/trajectories.json'):
    content = json.load(open(dataset_path))
    qualified_trajectories = [T for T in content if T['solved'] == True]
    train_trajectories, test_trajectories = train_test_split(qualified_trajectories, test_size=0.2, random_state=42)
    train_dataset = Dataset.from_list(train_trajectories)
    test_dataset = Dataset.from_list(test_trajectories)
    return train_dataset, test_dataset

def load_model_and_tokenizer(
    model_name: str = "Qwen/Qwen2.5-7B-Instruct",
    max_seq_length: int = 20000,
    dtype: Optional[torch.dtype] = None,
    load_in_4bit: bool = True,
    load_in_8bit: bool = False,
    r: int = 16,
    lora_alpha: int = 32,
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    """
    Load a model and tokenizer with specified configurations.
    
    Args:
        model_name: Name or path of the model to load
        max_seq_length: Maximum sequence length for the model
        dtype: Data type for model weights
        load_in_4bit: Whether to load the model in 4-bit quantization
        load_in_8bit: Whether to load the model in 8-bit quantization
    Returns:
        Tuple containing the loaded model and tokenizer
    """

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
        load_in_8bit=load_in_8bit
    )

    model = FastLanguageModel.get_peft_model(
            model,
            r = r, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                            "gate_proj", "up_proj", "down_proj",],
            lora_alpha = lora_alpha,
            lora_dropout = 0, # Supports any, but = 0 is optimized
            bias = "none",    # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
            random_state = 3407,
            use_rslora = False,  # We support rank stabilized LoRA
            loftq_config = None, # And LoftQ
            )
    
    return model, tokenizer

In [20]:


project_name = 'Wordle-Qwen2.5-7B'
run_name = 'UnslothFineTune_Qwen2.5-7B_Run1_4bit'

train_dataset, test_dataset = load_data()

model_name = "/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-7B-Instruct"
load_in_8bit = False
load_in_4bit = True
dtype=torch.bfloat16 if is_bfloat16_supported() else torch.float16
max_seq_length = 20000
r = 16
lora_alpha = 32
print(f"Loading model with r={r} and lora_alpha={lora_alpha}")

# Load model and tokenizer

model, tokenizer = load_model_and_tokenizer(
                                            model_name= model_name,
                                            max_seq_length= max_seq_length,
                                            dtype=dtype,
                                            load_in_4bit=load_in_4bit,
                                            load_in_8bit=load_in_8bit,
                                            r=r,
                                            lora_alpha=lora_alpha)

EOS_TOKEN = tokenizer.eos_token

Loading model with r=16 and lora_alpha=32
==((====))==  Unsloth 2025.6.5: Fast Qwen2 patching. Transformers: 4.52.4. vLLM: 0.9.1.
   \\   /|    NVIDIA A10G. Num GPUs = 1. Max memory: 22.184 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.6.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [103]:
def mask_target(target, seq):
    for i in range(len(seq) - len(target) + 1):
        if seq[i:i + len(target)] == target:
            seq[i:i + len(target)] = [-100] * len(target)
    return seq

def tokenize_dialog(dialog, tokenizer, EOS_TOKEN, max_seq_length):
    """
    Tokenizes a full dialog using Qwen's chat template, and masks non-assistant tokens.
    """
    # 1. Render the full conversation using the chat template
    EOT_ID = tokenizer.encode(EOS_TOKEN, add_special_tokens=False)[0]
    rendered_text = tokenizer.apply_chat_template(dialog, tokenize=False, add_generation_prompt=False)

    # 2. Tokenize that text (this returns a dict with 'input_ids', etc.)
    tokenized = tokenizer(rendered_text, return_tensors=None, add_special_tokens=False)
    
    # Cuda OOM issues, so we truncate the text all the way down to 128 tokens
    # tokenized = tokenizer(rendered_text, return_tensors=None, add_special_tokens=False, truncation=True, max_length=max_seq_length)
    
    input_ids = tokenized["input_ids"]

    # 3. Copy for label masking
    labels = copy.deepcopy(input_ids)

    # 4. Find indices of <|eot_id|> tokens (EOT)
    eot_indices = [i for i, token in enumerate(input_ids) if token == EOT_ID]

    # 5. Get role tokens
    system_token = tokenizer.encode("system", add_special_tokens=False)[-1]
    user_token = tokenizer.encode("user", add_special_tokens=False)[-1]

    labels[0] = -100  # Mask BOS or first token

    last_idx = None
    for idx in eot_indices:
        if last_idx is None:
            role_token = input_ids[1]
            last_idx = -1
        else:
            role_token = input_ids[last_idx+3] if last_idx+3 < len(input_ids) else None
        if role_token in (system_token, user_token):
            labels[last_idx + 1: idx + 2] = [-100] * (idx + 2 - last_idx -1)
        last_idx = idx
    
    # 6. Mask assistant headers
    assistant_header = tokenizer.encode("<|im_start|>assistant\n", add_special_tokens=False)
    labels = mask_target(assistant_header, labels)

    return {
        "input_ids": torch.tensor(input_ids, device='cuda'),
        "labels": torch.tensor(labels, device='cuda'),
        "attention_mask": torch.tensor([1] * len(input_ids), device='cuda'),
        "text": rendered_text
    }

In [112]:
max_len = 0
for i in range(len(train_dataset)):
    processed = tokenize_dialog(train_dataset[i]['messages'], tokenizer, EOS_TOKEN, max_seq_length)
    if len(processed['input_ids']) > max_len:
        max_len = len(processed['labels'])
print(max_len)

2409


In [105]:
processed['text']

'<|im_start|>system\nYou are a Wordle solver assistant. Your goal is to correctly guess the hidden 5-letter word within six attempts. The game provides feedback after each guess, which you must use to refine your subsequent guesses. Follow these rules and strategies carefully:\n\n== Wordle Rules ==\n1. The target word is exactly 5 letters long and is a valid English word.\n2. After each guess, you will receive feedback consisting of a string of 5 characters using the letters:\n   - G (Green): Correct letter in the correct position.\n   - Y (Yellow): Correct letter in the wrong position.\n   - B (Black): Letter is not in the target word at all.\n3. Use the feedback to eliminate invalid words and improve future guesses.\n\n== Feedback Interpretation Examples ==\n- Target word: "hello"\n- Guess: "world"\n- Feedback: "BYBGB"\n  Explanation:\n    - "w" → B: not in the target word\n    - "o" → Y: in the word, but not in position 2\n    - "r" → B: not in the word\n    - "l" → G: correct lette

In [106]:
non_masked_labels = []
for label in processed['labels']:
    if label != -100:    
        print(label, tokenizer.decode(label))
    else:
        print('MASKED')

MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED
MASKED

In [34]:
train_dataset[0]['messages']

[{'content': 'You are a Wordle solver assistant. Your goal is to correctly guess the hidden 5-letter word within six attempts. The game provides feedback after each guess, which you must use to refine your subsequent guesses. Follow these rules and strategies carefully:\n\n== Wordle Rules ==\n1. The target word is exactly 5 letters long and is a valid English word.\n2. After each guess, you will receive feedback consisting of a string of 5 characters using the letters:\n   - G (Green): Correct letter in the correct position.\n   - Y (Yellow): Correct letter in the wrong position.\n   - B (Black): Letter is not in the target word at all.\n3. Use the feedback to eliminate invalid words and improve future guesses.\n\n== Feedback Interpretation Examples ==\n- Target word: "hello"\n- Guess: "world"\n- Feedback: "BYBGB"\n  Explanation:\n    - "w" → B: not in the target word\n    - "o" → Y: in the word, but not in position 2\n    - "r" → B: not in the word\n    - "l" → G: correct letter and p

In [107]:
import ast

In [108]:
type(train_dataset[0]['messages'])

list

In [2]:
import json

content = json.load(open('/home/ubuntu/Wordle/English Words Dictionary.json'))

In [4]:
len(list(content.keys()))

370100

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-7B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-7B-Instruct")

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
prompt = tokenizer.apply_chat_template(messages, tokenize = False,add_generation_prompt=False)
print(prompt)








<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>



In [8]:
prompt

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nWhat is the capital of France?<|im_end|>\n'

In [2]:
from vllm import LLM
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = '/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-0.5B'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)




llm = LLM(
                    model=model.name_or_path,
                    tensor_parallel_size=1,
                    gpu_memory_utilization=0.3,
                    max_num_seqs=2,
                    max_model_len=4092,
                    # distributed_executor_backend="external_launcher",
                    # Feed identical seed for tp groups to ensure sampling results are the same across workers
                    seed=1337,
                    # Latest vLLM v1 memory profiler is misled by the high default value (i.e., 32768) - thinking there's not enough memory
                    max_num_batched_tokens=4096,
                )

INFO 06-29 15:18:18 [config.py:823] This model supports multiple tasks: {'generate', 'classify', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 06-29 15:18:18 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 06-29 15:18:20 [core.py:455] Waiting for init message from front-end.
INFO 06-29 15:18:20 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-0.5B', speculative_config=None, tokenizer='/home/ubuntu/TauBench/tau-bench/Model Tuning/Qwen2.5-0.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4092, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(back

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-29 15:18:22 [default_loader.py:272] Loading weights took 0.24 seconds
INFO 06-29 15:18:22 [gpu_model_runner.py:1624] Model loading took 0.9264 GiB and 0.447787 seconds
INFO 06-29 15:18:29 [backends.py:462] Using cache directory: /home/ubuntu/.cache/vllm/torch_compile_cache/63e485f994/rank_0_0 for vLLM's torch.compile
INFO 06-29 15:18:29 [backends.py:472] Dynamo bytecode transform time: 6.71 s
INFO 06-29 15:18:33 [backends.py:161] Cache the graph of shape None for later use
INFO 06-29 15:18:51 [backends.py:173] Compiling a graph for general shape takes 21.61 s
INFO 06-29 15:18:57 [monitor.py:34] torch.compile takes 28.32 s in total
INFO 06-29 15:18:59 [gpu_worker.py:227] Available KV cache memory: 5.52 GiB
INFO 06-29 15:18:59 [kv_cache_utils.py:715] GPU KV cache size: 482,304 tokens
INFO 06-29 15:18:59 [kv_cache_utils.py:719] Maximum concurrency for 4,092 tokens per request: 117.75x
INFO 06-29 15:19:24 [gpu_model_runner.py:2048] Graph capturing finished in 25 secs, took 0.38 GiB

In [7]:
messages =[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
]






llm.generate(messages)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: inputs must be a string, TextPrompt, TokensPrompt, or EmbedsPrompt